# 1. 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import json

import requests
import glob
import os

import urllib.request
from PIL import Image
import time
from tqdm import tqdm

# 2. 데이터 로드

In [2]:
df = pd.read_csv("Total_Data_1.csv")

# 3. OCR을 적용된 결과 JSON 파일로 만들기

In [9]:
df.reset_index(inplace=True)

* 쿠팡 파트너스에 대해서

In [10]:
df1 = df[df["광고 분류2"]==1].reset_index(drop=True)
df1.head(1)

,index,url,title,post_date,검색어,브랜드 명,content,content_cnt,content_line,quote,...,♡ 키워드,♥ 키워드,구매후기 키워드,한달 키워드,두달 키워드,세달 키워드,WEB_광고 분류1,WEB_광고 분류2,광고 분류1,광고 분류2
0,9,https://blog.naver.com/anfwjdtp/222565268273,가정용 전자동 커피머신★빈프레소 강제(?) 사용 후기ㅋ,20211111,빈프레소 리뷰,빈프레소,#가정용전자동커피머신 #가정용에스프레소머신 #가정용원두커피머신 #전자동커피머신 #빈...,1870,17,['가정용 전자동 커피머신★빈프레소 강제(?) 사용 후기 '],...,0,0,0,0,0,0,1,1,1,1


* 협찬, 체험단 등 소정료를 받아 작성된 광고성 블로그 리뷰

In [11]:
df2 = df[df["광고 분류2"]==2].reset_index(drop=True)
df2.head(1)

,index,url,title,post_date,검색어,브랜드 명,content,content_cnt,content_line,quote,...,♡ 키워드,♥ 키워드,구매후기 키워드,한달 키워드,두달 키워드,세달 키워드,WEB_광고 분류1,WEB_광고 분류2,광고 분류1,광고 분류2
0,0,https://blog.naver.com/PostView.naver?blogId=e...,무선 무릎찜질기 메이크맨 무르비2 사용 후기,20220124,무르비2 리뷰,메이크맨,저는 여름보다 유달리 겨울 추위를 잘 타는 체질이라 ... 히터가 작동되는 사무실에...,2435,22,['편리한 무선 온열찜질기진동 마사지 기능옥이 들어있어 원적외선 효과'],...,0,0,0,0,0,0,0,0,1,2


* 소정료를 받지 않고 직접 구매하여 작성된 솔직한 블로그 리뷰

In [12]:
df0 = df[df["광고 분류2"]==0].reset_index(drop=True)
df0.head(1)

,index,url,title,post_date,검색어,브랜드 명,content,content_cnt,content_line,quote,...,♡ 키워드,♥ 키워드,구매후기 키워드,한달 키워드,두달 키워드,세달 키워드,WEB_광고 분류1,WEB_광고 분류2,광고 분류1,광고 분류2
0,7,https://blog.naver.com/PostView.naver?blogId=i...,"빈프레소 커피머신, 가정용 전자동 커피머신 후기",20220304,빈프레소 리뷰,빈프레소,남편이 우리집 다음 커피머신기는 빈프레소라면서 빈프레소가 요즘 핫하다면서 그런데 비...,2329,25,['빈프레소 커피머신'],...,1,0,0,0,0,0,0,0,0,0


## 3.1 저장할 데이터 폴더 구축

In [13]:
import os
from tqdm import tqdm

os.makedirs("Images",exist_ok=True)
os.makedirs("Images/0",exist_ok=True)
os.makedirs("Images/1",exist_ok=True)
os.makedirs("Images/2",exist_ok=True)

os.makedirs("annotation",exist_ok=True)
os.makedirs("annotation/0",exist_ok=True)
os.makedirs("annotation/1",exist_ok=True)
os.makedirs("annotation/2",exist_ok=True)

In [15]:
df1.shape[0], df2.shape[0],df0.shape[0]

(172, 373, 1980)

* 쿠팡 파트너스 : 172
* 협찬을 받아 작성된 글 : 373  
* 협찬을 받지 않고 작성된 글 : 1980

## 3.2 json 파일로 저장

### 3.2.1 1번 -> 쿠팡 파트너스는 일정 수수료 받는다 (광고)

In [16]:
for i in tqdm(range(df1.shape[0])):
    if len(str(df1.iloc[i]["last_img"])) !=3: 
        try:
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
            urllib.request.install_opener(opener)
            
            save_number = df1.iloc[i]['index']
            save_img_path = f"./Images/1/{save_number}_class1.png"
            save_json_path = f"./annotation/1/{save_number}_class1.json"
            
            # 이미지 저장.
            urllib.request.urlretrieve(df1.iloc[i]["last_img"], save_img_path)
            
            ### 이미지 글자 추출
            time.sleep(3)
            files = {"file": open(save_img_path, "rb").read()}
            response = requests.post('http://27.255.77.102:5000/evaluation',files=files)
            if response.status_code == 200:
                feature_text = response.json()
                if len(feature_text) ==0:
                    print(save_number,"내용이 없다고? !!")
                    print(feature_text)
                    print()
                    continue
                else:
                    json_files = {"file": open(save_img_path, "rb").read()}
                    with open(save_json_path,'w',  encoding='utf-8') as f:
                        json.dump(feature_text, f, ensure_ascii=False, indent='\t')
        except:
            pass

  2%|█▉                                                                                | 4/172 [00:15<10:49,  3.87s/it]

36 내용이 없다고? !!
{}



  3%|██▊                                                                               | 6/172 [00:23<10:46,  3.90s/it]

48 내용이 없다고? !!
{}



 42%|█████████████████████████████████▉                                               | 72/172 [04:27<06:25,  3.85s/it]

...

 91%|████████████████████████████████████████████████████████████████████████▌       | 156/172 [10:02<01:05,  4.06s/it]

2394 내용이 없다고? !!
{}



 98%|██████████████████████████████████████████████████████████████████████████████▌ | 169/172 [10:53<00:12,  4.10s/it]

2460 내용이 없다고? !!
{}



100%|████████████████████████████████████████████████████████████████████████████████| 172/172 [11:05<00:00,  3.87s/it]


### 3.2.2 2번 -> 체험단, 협찬 , 소정료 받아서 씀(광고)

In [17]:
for i in tqdm(range(df2.shape[0])):
    if len(str(df2.iloc[i]["last_img"])) !=3: 
        try:
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
            urllib.request.install_opener(opener)
            
            save_number = df2.iloc[i]['index']
            save_img_path = f"Images/2/{save_number}_class2.png"
            save_json_path = f"annotation/2/{save_number}_class2.json"
            
            # 이미지 저장.
            urllib.request.urlretrieve(df2.iloc[i]["last_img"], save_img_path)
            
            ### 이미지 글자 추출
            time.sleep(1)
            files = {"file": open(save_img_path, "rb").read()}
            response = requests.post('http://27.255.77.102:5000/evaluation',files=files)
            if response.status_code == 200:
                feature_text = response.json()
                if len(feature_text) ==0:
                    print(save_number,"내용이 없다고? !!")
                    print(feature_text)
                    print()
                    continue
                else:
                    json_files = {"file": open(save_img_path, "rb").read()}
                    with open(save_json_path,'w',  encoding='utf-8') as f:
                        json.dump(feature_text, f, ensure_ascii=False, indent='\t')
        except:
            pass

  1%|▋                                                                                 | 3/373 [00:09<17:58,  2.91s/it]

2 내용이 없다고? !!
{}



  1%|▉                                                                                 | 4/373 [00:11<14:37,  2.38s/it]

3 내용이 없다고? !!
{}



  4%|███▎                                                                             | 15/373 [00:40<19:32,  3.28s/it]

...

 91%|████████████████████████████████████████████████████████████████████████▋       | 339/373 [12:12<01:07,  1.99s/it]

2385 내용이 없다고? !!
{}



 92%|█████████████████████████████████████████████████████████████████████████▊      | 344/373 [12:22<00:59,  2.04s/it]

2422 내용이 없다고? !!
{}



100%|████████████████████████████████████████████████████████████████████████████████| 373/373 [13:20<00:00,  2.15s/it]


### 3.2.3 일반 사용자들이 수수료나 협찬 등 제외하고 직접 돈 주고 쓴 후기들 (진솔한 리뷰)

In [18]:
import urllib.request
from PIL import Image
import time
from tqdm import tqdm

for i in tqdm(range(df0.shape[0])):
    if len(str(df0.iloc[i]["last_img"])) !=3: 
        try:
            save_number = df0.iloc[i]['index']
            save_img_path = f"Images/0/{save_number}_class0.png"
            save_json_path = f"annotation/0/{save_number}_class0.json"
            
            # 이미지 저장.
            urllib.request.urlretrieve(df0.iloc[i]["last_img"], save_img_path)
            
            ### 이미지 글자 추출
            time.sleep(1)
            files = {"file": open(save_img_path, "rb").read()}
            response = requests.post('http://27.255.77.102:5000/evaluation',files=files)
            if response.status_code == 200:
                feature_text = response.json()
                if len(feature_text) ==0:
                    print(save_number,"내용이 없다고? !!")
                    print(feature_text)
                    print()
                    continue
                else:
                    json_files = {"file": open(save_img_path, "rb").read()}
                    with open(save_json_path,'w',  encoding='utf-8') as f:
                        json.dump(feature_text, f, ensure_ascii=False, indent='\t')
        except:
            pass

  0%|▏                                                                              | 5/1980 [00:10<1:08:28,  2.08s/it]

15 내용이 없다고? !!
{}



  1%|▋                                                                             | 18/1980 [00:38<1:02:13,  1.90s/it]

34 내용이 없다고? !!
{}



  1%|▉                                                                               | 23/1980 [00:47<56:59,  1.75s/it]

...

2504 내용이 없다고? !!
{}



100%|███████████████████████████████████████████████████████████████████████████▊| 1976/1980 [1:08:01<00:06,  1.56s/it]

2517 내용이 없다고? !!
{}



100%|████████████████████████████████████████████████████████████████████████████| 1980/1980 [1:08:09<00:00,  2.07s/it]

2524 내용이 없다고? !!
{}



## 3.3 json 파일 정보를 DataFrame에 매핑

In [26]:
# json 파일을 매핑할 키 정보들
df[['광고 분류2',"index"]]

,광고 분류2,index
0,2,0
1,2,1
2,2,2
3,2,3
4,2,4
...,...,...
2520,2,2520
2521,1,2521
2522,2,2522
2523,0,2523


In [40]:
# jsonFilea 정보 저장
def json_labeling(x):
    index_num = int(x[0])
    label_num = int(x[1])
    load_name = f"annotation/{label_num}/{index_num}_class{label_num}.json" 
    return load_name

In [42]:
df["jsonFile"] = df[["index","광고 분류2"]].apply(lambda x: json_labeling(x), axis=1)

In [43]:
# 정보 확인
df

,index,url,title,post_date,검색어,브랜드 명,content,content_cnt,content_line,quote,...,♥ 키워드,구매후기 키워드,한달 키워드,두달 키워드,세달 키워드,WEB_광고 분류1,WEB_광고 분류2,광고 분류1,광고 분류2,jsonFile
0,0,https://blog.naver.com/PostView.naver?blogId=e...,무선 무릎찜질기 메이크맨 무르비2 사용 후기,20220124,무르비2 리뷰,메이크맨,저는 여름보다 유달리 겨울 추위를 잘 타는 체질이라 ... 히터가 작동되는 사무실에...,2435,22,['편리한 무선 온열찜질기진동 마사지 기능옥이 들어있어 원적외선 효과'],...,0,0,0,0,0,0,0,1,2,annotation/2/0_class2.json
1,1,https://blog.naver.com/PostView.naver?blogId=s...,REVIEW｜무릎 온열찜질기 &quot;메이크맨 무르비2&quot; 무릎찜질기 후기,20220129,무르비2 리뷰,메이크맨,최근 들어 아부지가 무릎이 자주 아프다고 하셔서 집에서 편하게 통증을 줄일 수 있는...,872,10,[],...,0,0,0,0,0,0,0,1,2,annotation/2/1_class2.json
2,2,https://blog.naver.com/PostView.naver?blogId=c...,메이크맨 무르비 2세대 무릎마사지기’ 솔직후기,20211125,무르비2 리뷰,메이크맨,"원적외선 무릎마사지기(온열/진동) SH-14K 76,900원 ➡️ 49,000원 요...",1893,21,['•무릎건강 자가 테스트•'],...,0,0,0,0,0,0,0,1,2,annotation/2/2_class2.json
3,3,https://blog.naver.com/PostView.naver?blogId=p...,원적외선 무릎마사지기 무르비2 무릎안마기 무릎찜질기 사용후기,20211009,무르비2 리뷰,메이크맨,남편이 무릎이 안 좋아서 이것저것 무릎 보호대만 엄청 사서 썼는데 쓰던것 찾아보니...,1083,23,"[""'해당 포스팅은 업체에게 제품을 제공받아 주관적인 소견으로 작성하였습니다.'""]",...,0,0,0,0,0,0,0,1,2,annotation/2/3_class2.json
4,4,https://blog.naver.com/PostView.naver?blogId=a...,무릎마사지와 무릎찜질이 가능한 무르비2 후기,20220120,무르비2 리뷰,메이크맨,저희 친정아빠가 술을 끊은지 몇년이 지났는데 작년 연말에 저희 남편이랑 갑자기 술을...,1188,16,[],...,0,0,0,0,0,0,0,1,2,annotation/2/4_class2.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520,2520,https://blog.naver.com/PostView.naver?blogId=h...,클럭 미니 마사지기 무릎 밴드 | 용도별 패드 체험 후기,20211030,무릎마사지기 리뷰,클럭,"얼마 전 클럭 미니 마사지기 발 패드, 대왕 패드, 큰 패드에 대한 소개를 간...",1389,15,[],...,0,0,0,0,0,0,0,1,2,annotation/2/2520_class2.json
2521,2521,https://blog.naver.com/PostView.naver?blogId=g...,웃챠 무릎마사지기 저주파마사지기 추천 솔직후기,20200324,무릎마사지기 리뷰,클럭,안녕하세요 오늘 소개해드릴 제품은 #웃챠 #무릎마사지기 입니다! 제가 정말 다양...,1739,20,[],...,0,0,0,0,0,0,0,1,1,annotation/1/2521_class1.json
2522,2522,https://blog.naver.com/PostView.naver?blogId=b...,추석 선물세트로 준비해 본 효도 선물 클럭 무릎 마사지기 리뷰,20210831,무릎마사지기 리뷰,클럭,안녕하세요! 뷰스타 쪼에요 : ) 지난 8월은 눈코 뜰 새 없이 바쁜 한 달을 보냈...,2740,20,[],...,0,0,1,0,0,0,0,1,2,annotation/2/2522_class2.json
2523,2523,https://blog.naver.com/PostView.naver?blogId=l...,[리뷰] 클럭(KLUG) 미니 마사지기 무릎 밴드,20200706,무릎마사지기 리뷰,클럭,안녕하세요. 오늘은 클럭 미니 마사지기 무릎 밴드 구매 후기에요. 지난번 5월 가정...,572,12,"['클럭 무릎 밴드2020.06.27', '끝']",...,0,0,0,0,0,0,0,0,0,annotation/0/2523_class0.json


## 3.4 json 파일을 읽어서 한 문장으로 만들기

In [70]:
# json 파일을 읽어서 OCR 결과를 하나의 텍스트 문장으로 만들기
def json_featuring(x):
    load_name  = x.values[0]
    try:
        with open(load_name, 'r', encoding='UTF-8') as json_file:
            json_data = json.load(json_file)
        result = " ".join(list(json_data.keys()))
    except FileNotFoundError:
        result = ""
    return result

In [71]:
df["context_img"] = df[["jsonFile"]].apply(lambda x: json_featuring(x), axis=1)

In [78]:
df[df["context_img"].str.contains("보스팅")]

,index,url,title,post_date,검색어,브랜드 명,content,content_cnt,content_line,quote,...,구매후기 키워드,한달 키워드,두달 키워드,세달 키워드,WEB_광고 분류1,WEB_광고 분류2,광고 분류1,광고 분류2,jsonFile,context_img
1,1,https://blog.naver.com/PostView.naver?blogId=s...,REVIEW｜무릎 온열찜질기 &quot;메이크맨 무르비2&quot; 무릎찜질기 후기,20220129,무르비2 리뷰,메이크맨,최근 들어 아부지가 무릎이 자주 아프다고 하셔서 집에서 편하게 통증을 줄일 수 있는...,872,10,[],...,0,0,0,0,0,0,1,2,annotation/2/1_class2.json,보스팅은 솔직하게 업체로부터 온 우 작성된 제공밭아 제품을 직접 체험 후기입니다.


In [81]:
# 정보 확인
for k in df["context_img"]:
    print(k)


보스팅은 솔직하게 업체로부터 온 우 작성된 제공밭아 제품을 직접 체험 후기입니다.


powe
a ce 칠옥타브
브랜드로부터 소정의 원고료를 이 작성되었습니다 지원받아 포스팅은 해당
beanpressi lon 행궁
1013 4.5점 a choice mare oo oro rome talv tazza travelers via 가능 결제 공식홈페이지 로마 리뷰 맛있는 목사 사진 시설 시용카드 이용 이지도 이탐리아 전체 주변 지역 커프 킴시0 트립어드바이저 평수
a the 구입 방을 사용한 수도 수수료를 실제 일환으로 있습니다. 제공 쿠팡파트너스 포스팅은 활동의 후 후기이자,
내림으
100x 검사 기는 기로 남음수 내부어 드리겠습니다 룸엄루이 모든 비프레소는 빈프레소는 사용하셔도 사용한 생기는 실시 아닐 안심하고 압습니다 에스티들 이슨니다 이하 있습니다 잔유물이오니 전수 전수검사(커프 전수검사를 전자동 제품 제품을 주조 진행하 책임 출고전 커피머신이 키스트 판마 패문어 하고 하기 하지 환불 환불제 휠니다 흐적(커피가루
andan pr

아이플르어 지사진

i테테크 각사한니다스 검색하고 꽃잔 네이버에서 늘러 늘러주시면 받아보실 배너를 빠르게 소식과 수 이벤트를 있어요 재미난 전문창작자 팬하기 팬하기틀
hing olid
beanpresso
th

oteat
beanpresso 블레드c 블렌드g 빈프레소 빈프레스 시그니처

anpresso pressi
11 1점 2점 3점 4점 5점
axcuve com macguyver naver 동영상 리뷰, 문의 제작
there thod
and c7
sorongersong
이사
so
d양 검색하고 네이버에서 리빙 전문들로거 팬하기
smisund

and helinox

an any as fre is kaviard nireditcol oj마켓 old rald resheasyeck sau thefl tot ure waw wwiv wwwastvecom 가기 눈 랭킹닭컴 마이셰프 무원 쇼핑하러 식품사 앙트레 오뚜기몰 정원e샵 직영몰 캐비아몰


httos instax liblo 결혼여행마집데이트 콩'sdaily
instax

instax minill rangi


10p 124,210 1개 단일상품 래덕발송 랜덤발송 로켓와우 미니11 블러쉬 세트 수량. 원 인스탁스 저용가빙 트나무데코 포토액자 필름 핑크 하 하드앨법
lew the 이마트
an instaxlen instox
mini11 nstax palette uceshoti
발생탈 사음 수노 수의이 슬식 이스니나 파트너스활동으로 후기이시만
instax instaxlensoo mini11

the
and
instax mini11

ange instax minil1
dicewhite instax mini1 mini11
tod whar
dean domave hito instaxminis mypalette 스
bo'so ind iz soit 리어 사라고 장채기이
and corot
insingers
10 instax retro the
instax

se

the
instax
319 arf doint le nstax print 넓어진 담다 당신의 더 사진으로 세상을

color pu 라임락퍼플 미니11 블러쉬핑크 스카이블루 아이스화이트 인스탁스 쟈골그레이 전용 케이스

instores
instay

and min nsta
텔나

ko
instax
dingesters


00
12.244 2020

0.31 instax lens ocus range

mini1 꿈꾸는 미물 여자 오늘도 이
공감라 김너다 림이 및국 은빛바다 정거장

a

jo sarker
instax
2편 그남 그래도 삿다 실수로 쓰자... 웰름을 흑백
3시 ol the thin u uin vew 가이트 금사사7 네이버쇼핑 다론 더보기 동영상 려모 미니즈 미스탁스 법적고지미안나 사이트 시식in 어학사산 이미지 이트 이프루어서 인스닥스미 인스닥스미니죽 인스락스미 인스탁스 인스탁스미 즉석 즉석필름 지도 지필름 트윈팩
23
to 라이
'업무식 망도론 소음(큰소리) 시간미 압주민의 조식 조심! 취침



In [114]:
# 정보 수정
df.loc[2221,"광고 분류1"] =1
df.loc[2221,"광고 분류2"] =1
df.loc[121,"광고 분류1"] =2
df.loc[121,"광고 분류2"] =2

# 4. 전처리된 데이터 저장

In [82]:
df.to_csv("./Total_Data2.csv",encoding="utf-8-sig",index=False)